In [1]:
# train_nifty_lstm.py
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import os

# ---------- CONFIG ----------
DATA_PATH = "data/nifty_data.csv"
MODEL_PATH = "models/nifty_lstm_model.h5"
TICKER = "^NSEI"      # NIFTY 50 index symbol
EPOCHS = 25
SEQ_LEN = 60

# ---------- CREATE FOLDERS ----------
os.makedirs("data", exist_ok=True)
os.makedirs("models", exist_ok=True)

# ---------- DOWNLOAD DATA ----------
print("📥 Downloading NIFTY 50 data...")
data = yf.download(TICKER, start="2015-01-01", end=None)
data.to_csv(DATA_PATH)
print(f"✅ Data saved to {DATA_PATH}")

# ---------- PREPARE DATA ----------
df = data[["Close"]].values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)

X_train, y_train = [], []
for i in range(SEQ_LEN, len(scaled_data)):
    X_train.append(scaled_data[i-SEQ_LEN:i, 0])
    y_train.append(scaled_data[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print("🧩 Training data shape:", X_train.shape)

# ---------- BUILD MODEL ----------
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(64, return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

# ---------- TRAIN ----------
print("🚀 Training LSTM model...")
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=32, verbose=1)

# ---------- SAVE MODEL ----------
model.save(MODEL_PATH)
print(f"✅ Model saved to {MODEL_PATH}")

# ---------- EVALUATE ----------
train_pred = model.predict(X_train)
train_pred_inv = scaler.inverse_transform(train_pred)
y_train_inv = scaler.inverse_transform(y_train.reshape(-1, 1))

mse = tf.keras.losses.MeanSquaredError()(y_train_inv, train_pred_inv).numpy()
print(f"📊 Training MSE: {mse:.2f}")


📥 Downloading NIFTY 50 data...


/var/folders/wg/d1y3qvk103gdr7sntsqgtyb40000gn/T/ipykernel_14471/1902322350.py:24: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(TICKER, start="2015-01-01", end=None)
[*********************100%***********************]  1 of 1 completed

✅ Data saved to data/nifty_data.csv
🧩 Training data shape: (2610, 60, 1)



/Users/bdeepaksai/Documents/Programing/Python/NNDL/.venv1/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

...

 Total params: 52,033 (203.25 KB)

 Trainable params: 52,033 (203.25 KB)

 Non-trainable params: 0 (0.00 B)

🚀 Training LSTM model...
Epoch 1/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.0144
Epoch 2/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0021
Epoch 3/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.0018
Epoch 4/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0017
Epoch 5/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.0014
Epoch 6/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.0014
Epoch 7/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0011
Epoch 8/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0011
Epoch 9/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0011
Epoch 10/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0010
Epoch 11/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.0010
Epoch 12/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 9.4419e-04
Epoch 13/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0010
Epoch 14/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 9.2276e-04
Epoch 15/25
82/82 ━━━━━━━━━━━━━━━━━━━━

✅ Model saved to models/nifty_lstm_model.h5
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
📊 Training MSE: 95732.05


In [3]:
# train_nifty_lstm.py

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import os

# ---------- CONFIG ----------
DATA_PATH = "data/nifty_data.csv"
MODEL_PATH = "models/nifty_lstm_model.h5"
TICKER = "^NSEI"      # NIFTY 50 index symbol
EPOCHS = 25
SEQ_LEN = 60

# ---------- CREATE FOLDERS ----------
os.makedirs("data", exist_ok=True)
os.makedirs("models", exist_ok=True)

# ---------- DOWNLOAD DATA ----------
print("Downloading NIFTY 50 data...")
data = yf.download(TICKER, start="2015-01-01", end=None)
data.to_csv(DATA_PATH)
print(f"Data saved to {DATA_PATH}")

# ---------- PREPARE DATA ----------
df = data[["Close"]].values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)

X_train, y_train = [], []
for i in range(SEQ_LEN, len(scaled_data)):
    X_train.append(scaled_data[i-SEQ_LEN:i, 0])
    y_train.append(scaled_data[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print("Training data shape:", X_train.shape)

# ---------- BUILD MODEL ----------
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(64, return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

# ---------- TRAIN ----------
print("Training LSTM model...")
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=32, verbose=1)

# ---------- SAVE MODEL ----------
model.save(MODEL_PATH)
print(f"Model saved to {MODEL_PATH}")

# ---------- EVALUATE ----------
train_pred = model.predict(X_train)
train_pred_inv = scaler.inverse_transform(train_pred)
y_train_inv = scaler.inverse_transform(y_train.reshape(-1, 1))

mse = tf.keras.losses.MeanSquaredError()(y_train_inv, train_pred_inv).numpy()
print(f"Training MSE: {mse:.2f}")


from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# ---------- EVALUATE ----------
print("Evaluating model performance...")

train_pred = model.predict(X_train)

# Inverse transform to get real prices
train_pred_inv = scaler.inverse_transform(train_pred)
y_train_inv = scaler.inverse_transform(y_train.reshape(-1, 1))

# ---------- METRICS ----------
mse = mean_squared_error(y_train_inv, train_pred_inv)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train_inv, train_pred_inv)
r2 = r2_score(y_train_inv, train_pred_inv)

# ---------- PERCENTAGE ACCURACY ----------
mape = np.mean(np.abs((y_train_inv - train_pred_inv) / y_train_inv)) * 100
accuracy = 100 - mape

print("\nModel Performance on Training Data:")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R² Score: {r2:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Approx. Training Accuracy: {accuracy:.2f}%")


📥 Downloading NIFTY 50 data...


/var/folders/wg/d1y3qvk103gdr7sntsqgtyb40000gn/T/ipykernel_1734/2301577343.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(TICKER, start="2015-01-01", end=None)
[*********************100%***********************]  1 of 1 completed

✅ Data saved to data/nifty_data.csv
🧩 Training data shape: (2615, 60, 1)



/Users/bdeepaksai/Library/Caches/JetBrains/PyCharmCE2024.2/demo/PyCharmLearningProject/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

...

 Total params: 52,033 (203.25 KB)

 Trainable params: 52,033 (203.25 KB)

 Non-trainable params: 0 (0.00 B)

🚀 Training LSTM model...
Epoch 1/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.0156
Epoch 2/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0020
Epoch 3/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0016
Epoch 4/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0016
Epoch 5/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0012
Epoch 6/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0013
Epoch 7/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0012
Epoch 8/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0012
Epoch 9/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0012
Epoch 10/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 9.7459e-04
Epoch 11/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0010
Epoch 12/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 8.7434e-04
Epoch 13/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 9.0975e-04
Epoch 14/25
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 8.7999e-04
Epoch 15/25
82/82 ━━━━━━━━━━━━

✅ Model saved to models/nifty_lstm_model.h5
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
📊 Training MSE: 128779.24
📈 Evaluating model performance...
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step

📊 Model Performance on Training Data:
Mean Squared Error (MSE): 128779.2486
Root Mean Squared Error (RMSE): 358.8583
Mean Absolute Error (MAE): 277.9824
R² Score: 0.9959
Mean Absolute Percentage Error (MAPE): 1.91%
✅ Approx. Training Accuracy: 98.09%
